# Programming Project - Unit 2,4
*by Igor A. Brandão and Leandro Antonio Feliciano da Silva*

**Goals**
The purpose of this project is explore the following:

- Full content of the statistical part seen in the course;
- Graphs generation;
- Geolocation analysis and hypotheses should be explained in detail.

## Global Imports

Import the necessary libraries to handle 

- Geocoding;
- Maps;
- File input;
- Heatmap;
- Bokeh charts;
- Numpy library;
- Tqdm progress bar

In [ ]:
### Library necessary to run this IPython Notebook
!pip install geocoder
!pip install folium
!pip install tqdm

In [135]:
# Import pandas
import pandas as pd

# Import google geocoder
import geocoder as gc

# Import numpy library
import numpy as np

# Import folium heatmap
import folium
from folium.plugins import HeatMap

# Import tqdm progressing bar plugin
from tqdm import tqdm

# Import bokeh libraries
from bokeh.plotting import figure
from bokeh.charts import Bar, Histogram, Donut, BoxPlot, output_notebook, show
from bokeh.layouts import row
from bokeh.models import HoverTool
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend

## Data importing

This cell is responsible to import the dataSet from excel file and save it into a variable **xl**

In [136]:
# Import pandas
import pandas as pd

# Assign spreadsheet filename: file
file = 'imd_student_blind.xlsx'

# Load spreadsheet: xl
xl = pd.ExcelFile(file)

# Print sheet names
print(xl.sheet_names)

['Sheet1']


## Data printing

Here the dataFrame receive the dataSet from **Sheet1** --> *xl.parse(0)*

In [137]:
# Load a sheet into a DataFrame by index: df
df = xl.parse(0)

# Print the head of the DataFrame df
df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado


## 1) Geolocation handler section

Here in this section, we'll handle the geolocalization infos. The idea is converting the zipcode into latitude and longitude and export the new dataSet (as it requires a long time to perform this operation).

After that, it'll be possible to pin the students position and generate the heatMap.

In [138]:
# df["GeoCod"] = df["CEP"]
df['lat'], df['long'] = [0, 0]

# Print new df
df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,lat,long
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado,0,0
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado,0,0
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado,0,0
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,0,0
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,0,0


### *Warning: Do not process this cell again!*

We've already converted all CEP into *lat/long*, you just need to skip this cell.

Please, use the **py-students-blind-with-lat-long.csv** file to generate the maps and save your time and your processor ;)

In [34]:
# Retrieve the latitude and longitute related to each student
for i in tqdm(range(len(df))):
    str = df.loc[i,'CEP']
    g = gc.google(str)
    if g.lat != None:
        df.ix[i,'lat'] = g.lat
        df.ix[i,'long'] = g.lng
print('Geocoding complete!')


100%|██████████| 4842/4842 [48:14<00:00,  2.18it/s] 

Geocoding complete!


In [139]:
# Print df with latitude and longitude
df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,lat,long
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado,0,0
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado,0,0
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado,0,0
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,0,0
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,0,0


### Latitude and longitude export

In order to avoid unnecessary processing, we are exporting the data *lat/long* from google API to a .csv file.

In [140]:
# Export the new dataSet to csv (don't run this cell again)
df.to_csv('py-students-blind-with-lat-long.csv', encoding="utf-8")

### Please, proceed from here :)

In [141]:
# Read the generate csv
geodata1 = pd.read_csv('py-students-blind-with-lat-long.csv', encoding="utf-8", index_col=0)

In [142]:
# Retrieve the data related to geolocalization
geodata = geodata1.filter(['a_ID','CEP','lat','long'], axis=1)
geodata = geodata.rename(columns = {'a_ID':'Aluno'})

# Reset the index
geodata = geodata.reset_index()

# Remove the the previous index
geodata.drop(geodata.columns[0], axis=1, inplace=True)
geodata.head()

,Aluno,CEP,lat,long
0,0,59015430,-5.816641,-35.200015
1,0,59015430,-5.816641,-35.200015
2,1,59073120,-5.853337,-35.252804
3,2,59072580,-5.832998,-35.242542
4,3,59088150,-5.872282,-35.206600


### Pin map

The idea here is generating a map with pinnings indicating the student location

In [159]:
# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

# Create map object
map_osm = folium.Map(location=mapc, zoom_start=zoom)

# Plot each of the locations that we geocoded
for j in tqdm(range(len(geodata))):
    folium.Marker([geodata.ix[j,'lat'], geodata.ix[j,'long']],
                  #popup=(geodata.ix[j,'Aluno'])
                 ).add_to(map_osm)
# Show the map
map_osm

100%|██████████| 4842/4842 [01:12<00:00, 66.84it/s]


### Heatmap map

In order to see in a properly way the students concentration in Natal, we'll generate a hetmap to demonstrate that using colors.

Cold colors represent low concentration of students, and hot colors indicate high concentrations.

In [143]:
# Do some fix in final data
dataFinal = geodata.copy()
dataFinal["Count"] = 0

# Print the dataSet head
dataFinal.head()

,Aluno,CEP,lat,long,Count
0,0,59015430,-5.816641,-35.200015,0
1,0,59015430,-5.816641,-35.200015,0
2,1,59073120,-5.853337,-35.252804,0
3,2,59072580,-5.832998,-35.242542,0
4,3,59088150,-5.872282,-35.206600,0


In [144]:
# Cound the number of students by zipcode
dataFinal = pd.DataFrame(dataFinal.groupby(["CEP"])['Count'].count()).reset_index()

# Add latitude and longitude to dataFinal
dataFinal["lat"] = geodata['lat']
dataFinal["long"] = geodata['long']

# Print the dataSet head
dataFinal.head()

,CEP,Count,lat,long
0,0,289,-5.816641,-35.200015
1,1507000,6,-5.816641,-35.200015
2,5021000,6,-5.853337,-35.252804
3,5163000,7,-5.832998,-35.242542
4,6321200,9,-5.872282,-35.206600


In [160]:
# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

# Initialize the coordinates array
coordinates = []

# Add the coordinates to the coordinate
for i in range(len(dataFinal)):
    # eliminate items with'nan' element
    if all(~np.isnan([dataFinal.ix[i,'lat'], dataFinal.ix[i,'long'], dataFinal.ix[i,'Count']])):
        coordinates.append([dataFinal.ix[i,'lat'], dataFinal.ix[i,'long'], dataFinal.ix[i,'Count']])

# Create map object
htMap = folium.Map(location=mapc, zoom_start=zoom)

# Append the coordinates to the heatMap
HeatMap(coordinates).add_to(htMap)

# Print the heatMap
htMap

## 2) Statistic handler section

Here in this section, we'll handle the statistics infos.

### Grades histogram

Here we'll analyse the grades taking into consideration its distribution

In [145]:
# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Make the Histogram: p
hist_grade = Histogram(df, 'nota', title='Students grades distribution', 
              legend='top_left', tools=TOOLS, bins=50, 
                       background_fill_color="#E8DDCB", color="#036564")

# Set axis labels
hist_grade.xaxis.axis_label = 'Grades (0 to 10)'
hist_grade.yaxis.axis_label = 'Grades frequency'

# Call the output_notebook() 
output_notebook()
show(hist_grade)

Loading BokehJS ...

## Students' scores vs. total students

<p>As a first impression we see the large number of students that score 0, approximately 440 students were scored zero, this can be a consequence of several reasons, one of that could be the number of students dropped out of the discipline which favors the number of students with zero grade.

We can also see the number of students grades near to 10 is very low, and the biggest concentration of grades between 5 and 8.<p>

In [146]:
# Imports
import numpy as np
import scipy.special

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_notebook

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset, hover, save'

p1 = figure(title="Student grades - Normal Distribution (μ=0, σ=0.5)",tools=TOOLS,
            background_fill_color="#E8DDCB")

# Confidence interval
mu, sigma = 0, 0.5

# Histogram settings
hist, edges = np.histogram(df['nota'], density=True, bins=50)

# Tendency line settings
x = np.linspace(-5, 10, 1000)

# Probability density function
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))

# Cumulative density function
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")

# Add the lines
p1.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="Probability density function")
p1.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="Cumulative density function")

# Customs
p1.legend.location = "top_left"
p1.xaxis.axis_label = 'Grades (0 to 10)'
p1.yaxis.axis_label = 'Grades frequenc)'

# Print the distribution
output_notebook()
show(gridplot(p1, ncols=2, plot_width=800, plot_height=600, toolbar_location=None))

Loading BokehJS ...

In [148]:
from bokeh.palettes import RdBu3

# Receiva data from dataFrame
data = df.copy()

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Make a box plot: p
p = BoxPlot(data, values='nota', label='status.disciplina', color='status.disciplina',
            title='Student grades (grouped by discipline status)',
            legend='bottom_right', background_fill_color="#E8DDCB",
            plot_width=600, plot_height=600, tools=TOOLS)

# Set the y axis label
p.yaxis.axis_label='Student grades'

# Call the output_notebook() 
output_notebook()
show(p)

Loading BokehJS ...

## Grades from aproved vs reproved students

Through these two boxplots we can notice a huge difference on the median of the grades, while the students who were successful in the discipline were with averages around 6.7 the students who did not have had a very low median around 1.3. While among the approved students they took notes ranging from 5 to 10 among those who failed, we had scores ranging from 0 to a little below 5.

In [149]:
from bokeh.palettes import RdBu3

# Receiva data from dataFrame
data = df.copy()

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Make a box plot: p
p = BoxPlot(data, values='nota', label='status', color='status',
            title='Student grades (grouped by course status)',
            legend='bottom_right', background_fill_color="#E8DDCB",
            plot_width=600, plot_height=600, tools=TOOLS)

# Set the y axis label
p.yaxis.axis_label='Student grades'

# Call the output_notebook() 
output_notebook()
show(p)

Loading BokehJS ...

## Analysis of student grades versus course status

<p> The first boxplot shows the students' grades in the course, we can see that the average grade is around 5.6.
Already in the second boxplot of the students who canceled the course, we can see that the average of the grades turns around 2.07 and more than 75% of the students who canceled the course had grades below 5.7 very close to the median of the students . Already among the students who locked the discipline 75% of them, took grades below 7.0. </p>
<p> An interesting fact that we can observe is that among students who are categorized as completed, formed or forming, we do not observe 0 marks, and the minimum values ​​rotate around 4.0 to 5.2. </p>
<P> In all categories we can see outliers with scores of close to 10</p>

<p>Another note about students with completed, formed or graduating status is their median if they are all above the average that is 7.0. And 50% of the graduates had scores between 6.9 and 9</p>

In [150]:
from bokeh.palettes import RdBu3

# Receiva data from dataFrame
data = df.copy()

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Make a box plot: p
p = BoxPlot(data, values='nota', label='ano_disciplina', color='ano_disciplina',
            title='Student grades (grouped by discipline year)',
            legend='bottom_right', background_fill_color="#E8DDCB",
            plot_width=600, plot_height=600, tools=TOOLS)

# Set the y axis label
p.yaxis.axis_label='Student grades'

# Call the output_notebook() 
output_notebook()
show(p)

Loading BokehJS ...

## Student grades by discipline year Review

As a first observation we can see that from 2014 to 2016 we always had outliers with close scores of 10, but the same result was not observed yet in 2017, the boxplots also show that for medians from 2014 to 2016 the average of the scores was between 4, 7 and 6.0. As early as 2017 the average grade was close to 2.5.
For quartiles we see that at least 25% of the students had scores below 2.7, 2.2, 1.2 and 1.3 from 2014 to 2017 respectively.

In [151]:
# Do some fix in disciplineStatus
grades = df.copy()
grades["Count"] = 0

# Count the status sum-up
grades = pd.DataFrame(grades.groupby(["nota"])['Count'].count()).reset_index()

# Print the dataSet head
grades.head()

,nota,Count
0,0.0,401
1,0.1,43
2,0.2,59
3,0.3,60
4,0.4,32


### Student grades by range (0 to 10)

In [152]:
# Tools
TOOLS = 'box_zoom,box_select,resize,reset,hover,save'

# Grades < 3
grades1 = grades[grades["nota"] < 3]["nota"]
frequency1 = grades[grades["nota"] < 3]["Count"]

# 3 <= Grades <= 5
grades2 = grades[(3 <= grades["nota"]) & (grades["nota"] <= 5)]["nota"]
frequency2 = grades[(3 <= grades["nota"]) & (grades["nota"] <= 5)]["Count"]

# 5 <= Grades <= 8
grades3 = grades[(5 <= grades["nota"]) & (grades["nota"] <= 8)]["nota"]
frequency3 = grades[(5 <= grades["nota"]) & (grades["nota"] <= 8)]["Count"]

# 8 < Grades
grades4 = grades[grades["nota"] > 8]["nota"]
frequency4 = grades[grades["nota"] > 8]["Count"]

# Create the figure: p
p = figure(x_axis_label='Grades (per year)', y_axis_label='Grades frequency', tools=TOOLS)

# Add the data to the plot
p.circle(grades1, frequency1, color="#990990", size=10, alpha=0.8)
p.circle(grades2, frequency2, color="#990000", size=10, alpha=0.8)
p.circle(grades3, frequency3, color="#009900", size=10, alpha=0.8)
p.circle(grades4, frequency4, color="#000099", size=10, alpha=0.8)

# Call the output_notebook() 
output_notebook()

# Display the plot
show(p)

Loading BokehJS ...

### Student status analysis

The idea here is analyse the (%) between approvals and desaprovals

In [153]:
# Do some fix in studentStatus
studentStatus = df.copy()
studentStatus["Count"] = 0

# Print the dataSet head
studentStatus.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,lat,long,Count
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado,0,0,0
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado,0,0,0
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado,0,0,0
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,0,0,0
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,0,0,0


In [154]:
# Count the status sum-up
studentStatus = pd.DataFrame(studentStatus.groupby(["status.disciplina"])['Count'].count()).reset_index()

# Print the dataSet head
studentStatus.head()

,status.disciplina,Count
0,Aprovado,2766
1,Reprovado,2076


In [155]:
from bokeh.palettes import RdBu3

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Donut chart settings
d = Donut(studentStatus, label=['status.disciplina', 'Count'], values='Count',
          text_font_size='12pt', hover_text='status_count', legend='top_left', 
          tools=TOOLS, background_fill_color="#E8DDCB", title='Total approvals and desaprovals', 
          color=RdBu3)

# Print the chart
output_notebook()
show(d)

Loading BokehJS ...

## Approved vs. disapproved

Through this graph we can see that the percentage of students that fail in this discipline is almost equal to the total number of students approved approximately 42%.

### Discipline status analysis

Here we'll check the (%) of students that quit, cancel or get through the discipline

In [156]:
# Do some fix in disciplineStatus
disciplineStatus = df.copy()
disciplineStatus["Count"] = 0

# Print the dataSet head
disciplineStatus.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,lat,long,Count
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado,0,0,0
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado,0,0,0
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado,0,0,0
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,0,0,0
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,0,0,0


In [157]:
# Count the status sum-up
disciplineStatus = pd.DataFrame(disciplineStatus.groupby(["status"])['Count'].count()).reset_index()

# Print the dataSet head
disciplineStatus.head()

,status,Count
0,ATIVO,3440
1,CANCELADO,1047
2,CONCLUIDO,20
3,FORMADO,10
4,FORMANDO,43


In [158]:
from bokeh.palettes import RdBu9

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Donut chart settings
d = Donut(disciplineStatus, label=['status', 'Count'], values='Count',
          text_font_size='10pt', hover_text='status', legend='top_left', 
          tools=TOOLS, background_fill_color="#E8DDCB", title='Discipline status', 
          color=RdBu9, plot_width=600, plot_height=600)

# Print the chart
output_notebook()
show(d)

Loading BokehJS ...

## Distribution of student status

<p>When we look at the donut above we can see that the number of students who are graduates or trainees are very small compared to the other statuses as active, canceled and locked students.

An alarming fact is that a large percentage something like 15 to 20 percent of students cancel or lock up the discipline.</p>